# 研究データ管理計画を立てる

研究データ管理計画を立てるタスクです。<br>
Gakunin RDMで作成されているプロジェクトメタデータ（DMP）を取得、表示します。<br>
プロジェクトメタデータは複数登録できるため、表示されるプロジェクトメタデータのタイトルから取り込みたいものを選択してください。<br>
※プロジェクトメタデータが登録されていない場合は、Gakunin RDMに戻って登録を行ってください。<br>

## 研究データ管理計画（DMP）を取得する
研究データ管理計画（DMP）をGakunin RDMから取り込みます。<br>
Gakunin RDMでDMP（プロジェクトメタデータ）を登録していない場合は、Gakunin RDMに戻ってプロジェクトメタデータを作成してください。<br>

In [ ]:
# 研究データ管理計画（DMP）を取得する
import os
from pathlib import Path
import traceback

import panel as pn
from IPython.display import display, clear_output
from requests.exceptions import RequestException

from library.utils.config import message as msg_config
from library.task_director import TaskDirector
from library.utils.setting import  DMPManager
from library.utils.widgets import MessageBox, Button, Alert
from library.utils.storage_provider import grdm
from library.utils.error import (NotFoundContentsError, UnauthorizedError,
                                 ProjectNotExist, UnusableVault, PermissionError)
from library.utils.input import get_grdm_connection_parameters

script_file_name = "make_research_data_management_plan"
notebook_name = script_file_name+'.ipynb'
script_dir_path = os.path.dirname('__file__')
p = Path(script_dir_path)
# dmp.json(data_gorvernance\researchflow\plan\dmp.json)
dmp_file = p.joinpath('../../../../../researchflow/plan/dmp.json').resolve()

class DMPViewer:
    """DMPを取得し、保存するクラスです。

    Attributes:
        instance:
            dmp_path(str) : DMPのパス
            dmp_file(DMPManager) : DMPのファイル
            dmps(dict) : DMP
            dmp_selector(pn.widgets.Select) : DMPの選択
    """

    def __init__(self, dmp_path) -> None:
        """DMPViewer コンストラクタのメソッドです。

        Args:
            dmp_path(str) : DMPのパス
        """
        self.dmp_file = DMPManager(dmp_path)
        self.dmps = {}

        self.dmp_selector = pn.widgets.Select(width=600, size=5)

    def is_dmp_file(self):
        """DMPファイルが存在するか確認するメソッドです。

        Returns:
            bool:DMPのファイルが存在すればTrue,存在しなければFalseを返す。    
        """
        return self.dmp_file.path.is_file()

    ##### display registrated dmp #####
    def display_registrated_dmp(self):
        """保存されているDMPを表示するメソッドです。
        
        Returns:
            Alert:保存されているDMPのメッセージを返す。
        
        """
        dmp = self.dmp_file.read()
        obj = self.dmp_file.display_format(dmp)
        return Alert.info(obj)

    ##### select dmp #####
    def make_select_dmp_form(self, dmps):
        """DMPを選択するフォームを作成するメソッドです。

        Args:
            dmps(dict[str,list]):プロジェクトメタデータ

        Returns:
            dmp_selector(dict):DMPを選択するフォーム
        """
        options = dict()
        self.dmps = dmps
        options.update(self.dmp_file.create_dmp_options(self.dmps))

        self.dmp_selector.options = options
        self.dmp_selector.value = 0
        return self.dmp_selector

    def register_dmp(self)->str:
        """選択したDMPをファイルに保存する
        
        Returns:
            DMPManager:保存したDMPの結果を表示させる。
        """
        index = self.dmp_selector.value
        if not self.dmps:
            return ''
        dmp = self.dmp_file.get_dmp(self.dmps, index)
        self.dmp_file.write(dmp)
        return self.dmp_file.display_format(dmp)

class DGPlaner(TaskDirector):
    """フェーズ：研究準備、タスク：研究データ管理計画を立てるのコントローラークラスです。

    Attributes:
        instance:
            form_box(pn.WidgetBox):フォームを格納する。
            msg_output(MessageBox):ユーザーに提示するメッセージを格納する。
            viewer(DMPViewer):DMPViewerクラスを呼び出す。
            working_path (str): 実行Notebookファイルパス
    """

    def __init__(self, working_path:str) -> None:
        """DGPlaner コンストラクタのメソッドです。

        Args:
            working_path (str): 実行Notebookファイルパス
        """
        # working_path = .data_gorvernance/researchflow/plan/task/plan/make_research_data_management_plan.ipynbが想定値
        super().__init__(working_path, notebook_name)

        # フォームボックス
        self.form_box = pn.WidgetBox()
        self.form_box.width = 900
        # メッセージ用ボックス
        self.msg_output = MessageBox()
        self.msg_output.width = 900

        self.viewer = DMPViewer(dmp_file)

    def get_grdm_params(self):
        """GRDMのトークンとプロジェクトIDを取得するメソッドです。
        
        Returns:
            token(str):GRDMのトークンの値を返す。
            project_id(str):プロジェクトIDの値を返す。
        """
        token = ""
        project_id = ""
        try:
            token, project_id = get_grdm_connection_parameters()
        except UnusableVault:
            message = msg_config.get('form', 'no_vault')
            self.msg_output.update_error(message)
            self.log.error(traceback.format_exc())
        except PermissionError:
            message = msg_config.get('form', 'insufficient_permission')
            self.msg_output.update_error(message)
            self.log.error(traceback.format_exc())
        except ProjectNotExist as e:
            self.msg_output.update_error(str(e))
            self.log.error(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self.msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self.msg_output.update_error(message)
            self.log.error(message)
        return token, project_id

    def get_project_metadata(self, base_url, token, project_id):
        """プロジェクトメタデータを取得するメソッドです。

        Args:
            base_url (str):Root URL (e.g. https://rdm.nii.ac.jp)
            token (str): パーソナルアクセストークン
            project_id (str): プロジェクトID

        Returns:
            dmps[str,list]:プロジェクトメタデータの値を返す。
        """
        dmps = {}
        try:
            dmps = grdm.get_project_metadata(base_url, token, project_id)

        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self.msg_output.update_warning(message)
            self.log.warning(traceback.format_exc())
        except ProjectNotExist:
            message = msg_config.get('form', 'project_id_not_exist').format(project_id)
            self.msg_output.update_error(message)
            self.log.error(traceback.format_exc())
        except NotFoundContentsError:
            message = msg_config.get('make_research_data_management_plan', 'metadata_not_exist')
            self.msg_output.update_warning(message)
            self.log.warning(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self.msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        return dmps

    def create_select_dmp_form(self, base_url, token, project_id):
        """DMPを選択するフォームを作成するメソッドです。

        Args:
            base_url (str):Root URL (e.g. https://rdm.nii.ac.jp)
            token (str): パーソナルアクセストークン
            project_id (str): プロジェクトID
        """
        dmps = self.get_project_metadata(base_url, token, project_id)
        if dmps:
            self.form_box.clear()
            self.form_box.append(self.viewer.make_select_dmp_form(dmps))
            self.dmp_select_button = Button(width=500)
            msg = msg_config.get('make_research_data_management_plan', 'select')
            self.dmp_select_button.set_looks_init(msg)
            self.dmp_select_button.on_click(self._dmp_select_callback)
            self.form_box.append(self.dmp_select_button)

    @TaskDirector.task_cell("1")
    def get_dmp(self):
        """DMPを取得してその結果を表示するメソッドです。"""
        # タスク開始による研究準備のサブフローステータス管理JSONの更新
        self.doing_task()
        # フォーム定義
        try:
            self.token, self.project_id = self.get_grdm_params()
            self.base_url = grdm.BASE_URL
            clear_output()

            if not self.msg_output.has_message() and self.token and self.project_id:
                if self.viewer.is_dmp_file():
                    self.get_dmp_button = Button(width=500)
                    msg = msg_config.get('make_research_data_management_plan', 'get_dmp')
                    self.get_dmp_button.set_looks_init(msg)
                    self.get_dmp_button.on_click(self._get_other_dmp)
                    self.form_box.append(self.get_dmp_button)
                    self.form_box.append(self.viewer.display_registrated_dmp())
                else:
                    self.create_select_dmp_form(self.base_url, self.token, self.project_id)

        except Exception:
            message = msg_config.get('DEFAULT', 'unexpected_error')
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self.log.error(message)
            self.msg_output.update_error(message)

        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self.form_box)
        form_section.append(self.msg_output)
        display(form_section)

    @TaskDirector.callback_form('DMPを取得する')
    def _get_other_dmp(self, event):
        """dmpを取得するメソッドです。"""
        self.get_dmp_button.set_looks_processing()
        self.msg_output.clear()
        try:
            self.create_select_dmp_form(self.base_url, self.token, self.project_id)
        except Exception:
            message = msg_config.get('DEFAULT', 'unexpected_error')
            self.get_dmp_button.set_looks_error(message)
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self.log.error(message)
            self.msg_output.update_error(message)
            return

    @TaskDirector.callback_form('DMPを選択する')
    def _dmp_select_callback(self, event):
        """DMPを選択するメソッドです。"""
        self.dmp_select_button.set_looks_processing()
        self.msg_output.clear()
        try:
            metadata = self.viewer.register_dmp()
            self.msg_output.update_info(metadata)
            self.form_box.clear()
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self.log.error(message)
            self.msg_output.update_error(message)

DGPlaner(working_path=os.path.abspath('__file__')).get_dmp()

## データガバナンス機能追加設定を行う
データガバナンス機能に必要な追加設定を行います。<br>
追加設定によってこれ以降のタスクが推奨、非推奨に分別され、非推奨のタスクは各サブフローメニューの初期表示では選択できないようになります。<br>
（非推奨タスクでもサブフローメニューで「すべてのタスク」を選択すると表示され実行可能です。）

In [ ]:
# データガバナンス機能追加設定を行う
import os
import traceback
from typing import Any, List
from pathlib import Path
import json
import sys

import panel as pn
from panel.widgets import Checkbox
from IPython.display import display

sys.path.append('../../../../..')
from library.utils.config import path_config, message as msg_config
from library.task_director import TaskDirector
from library.utils.widgets import MessageBox
from library.utils.setting import get_dg_customize_config, SubflowStatusFile, SubflowStatus


script_file_name = "make_research_data_management_plan"
notebook_name = script_file_name+'.ipynb'
script_dir_path = os.path.dirname('__file__')
p = Path(script_dir_path)
# DGカスタマイズJSON定義書パス(data_gorvernance\library\data\data_governance_customize.json)
data_governance_customize_file = p.joinpath('../../../../..', 'library/data/data_governance_customize.json').resolve()


class DGCustomizeSetter(TaskDirector):
    """データガバナンス機能追加設定のフォーム定義、データの登録と更新、追加したデータガバナンス機能の表示を行うクラスです。  

    Attributes:
        instance:
            working_path(str):実行Notebookファイルパス
            _plan_path(str):JSON定義書のパス
            _form_box(pn.WidgetBox):フォームを格納する。
            _msg_output(MessageBox):ユーザーに提示するメッセージを格納する。
    """

    def __init__(self, working_path:str, plan_file:str) -> None:
        """DGCustomizeSetter コンストラクタのメソッドです。

        Args:
            working_path (str): 実行Notebookファイルパス
            plan_file (str): JSON定義書のパス
        """
        # working_path = .data_gorvernance/researchflow/plan/task/plan/make_research_data_management_plan.ipynbが想定値
        super().__init__(working_path, notebook_name)

        # α設定JSON定義書(plan.json)
        # 想定値：data_gorvernance\researchflow\plan\plan.json
        self._plan_path =  plan_file

        # フォームボックス
        self._form_box = pn.WidgetBox()
        self._form_box.width = 900
        # メッセージ用ボックス
        self._msg_output = MessageBox()
        self._msg_output.width = 900

    def define_form(self):
        """フォーム定義"""
        # α設定チェックボックスリスト
        self._checkbox_list = pn.Column()
        for id in self.get_data_governance_customize_ids():
            check_book = pn.widgets.Checkbox(name=msg_config.get('data_governance_customize_property', id))
            self._checkbox_list.append(check_book)
        self.submit_button = pn.widgets.Button(width=500)
        self.change_submit_button_init(name=msg_config.get('form', 'submit_select'))
        self.submit_button.on_click(self.callback_submit_input)
        # フォームボックスを更新
        self.update_form_box(
            title=msg_config.get('plan', 'form_title_set_data_governance_customize_property'), objects=[self._checkbox_list, self.submit_button]
        )


    def update_form_box(self, title:str, objects:List[Any]):
        """フォームボックスの更新をするメソッドです。

        Args:
            title (str): フォームのメッセージ
            objects (List[Any]): チェックボックスリスト、ボタン      
        """
        self._form_box.clear()
        self._form_box.append(f'## {title}')
        for object in objects:
            self._form_box.append(object)

    def get_plan_data(self)->dict:
        """JSON定義書のデータを取得するメソッドです。

        Returns:
            dict: JSON定義所のデータの値を返す。
        """
        plan_path = Path(self._plan_path)
        with plan_path.open('r') as file:
            return json.loads(file.read())

    def update_plan_data(self, data:dict):
        """JSON定義書のデータを更新するメソッドです。

        Args:
            data (dict): JSON定義書のデータ
        """
        plan_path = Path(self._plan_path)
        with plan_path.open('w') as file:
            file.write(json.dumps(data, indent=4))

    def callback_submit_input(self, event):
        """登録内容を出力するメソッドです。
               
           DGカスタマイズプロパティの設定値をa設定JSON定義書に記録し、その登録した内容を出力します。

        Raises:
            Exception:panel.widgets.Checkboxでないか、cbの値がbool型でないエラー
        """
        try:
            # 適応するDGカスタマイズプロパティの設定値をα設定JSON定義書(data_gorvernance\researchflow\plan\plan.json)に記録する。
            plan_data = self.get_plan_data()

            registration_content = ''
            for index, cb in enumerate(self._checkbox_list):
                if type(cb) is Checkbox and type(cb.value) is bool:
                    plan_data['governance_plan'][index]['is_enabled'] = cb.value
                    governance_plan_id = plan_data['governance_plan'][index]['id']
                    registration_content += f'{msg_config.get("data_governance_customize_property", governance_plan_id)} : {self.get_msg_disable_or_able(cb.value)}<br>'
                else:
                    raise Exception('cb variable is not panel.widgets.Checkbox or cb value is not bool type')

            self.update_plan_data(plan_data)
            # タスクの無効化処理
            self.disable_task_by_phase()

            # 登録内容を出力する
            registration_msg = f"""### {msg_config.get("form", "registration_content")}

<hr>

{registration_content}
            """
            self._msg_output.clear()
            alert = pn.pane.Alert(registration_msg, sizing_mode="stretch_width",alert_type='info')
            self._msg_output.append(alert)
            self.change_submit_button_success(msg_config.get('form', 'accepted'))
            # TODO: 開発中の仮置きのため後で削除すること
            self.done_task()

        except Exception as e:
            self._msg_output.clear()
            alert = pn.pane.Alert(f'## [INTERNAL ERROR] : {traceback.format_exc()}',sizing_mode="stretch_width",alert_type='danger')
            self._msg_output.append(alert)

    def get_msg_disable_or_able(self, b:bool)->str:
        """有効なメッセージか無効なメッセージを取得するメソッドです。

        Args:
            b (bool): チェックボックスリストの値

        Returns:
            str:有効なメッセージの値を返す。
            str:無効なメッセージの値を返す。
        """
        if b:
            return f'<span style="color: red; ">**{msg_config.get("DEFAULT", "able")}**</span>'
        else:
            return msg_config.get('DEFAULT', 'disable')

    def get_data_governance_customize_id_by_index(self, index)->str:
        """データガバナンスカスタマイズIDのデータを取得するメソッドです。

        Args:
            index (int): インデックス

        Returns:
            ids(str): データガバナンスカスタマイズIDの値を返す。
        """
        ids = self.get_data_governance_customize_ids()
        return ids[index]

    def get_data_governance_customize_data(self)->List[dict]:
        """データガバナンスカスタマイズのデータを取得するメソッドです。

        Returns:
            List[dict]: データガバナンスカスタマイズのデータの値を返す。
        """
        with data_governance_customize_file.open('r') as file:
            data_governance_customize_data = json.loads(file.read())
            return data_governance_customize_data['dg_customize']

    def get_data_governance_customize_ids(self)->List:
        """データガバナンスカスタマイズIDのデータを取得するメソッドです。

        Returns:
            List: データガバナンスカスタマイズIDの値を返す。
        """
        return [p['id'] for p in self.get_data_governance_customize_data()]

    def get_disable_task_ids_on_phase(self)->dict[str, list[str]]:
        """無効化（非表示:任意タスク）のタスクIDをフェーズごとに集計するメソッドです。

           JSON定義書の設定値を取得し、 DGカスタマイズプロパティの設定値とDGカスタマイズJSONデータから無効にするタスクIDを取得します。
        
        Returns:
            dict[str,list[str]]:無効化タスクIDデータの値を返す。
        """
        # α設定JSON定義書の設定値を取得
        plan_data = self.get_plan_data()
        # DGカスタマイズ定義データを取得する
        dg_customize_config = get_dg_customize_config()

        # 無効化タスクIDデータ
        disable_task_ids_on_phase:dict[str, list[str]] = {}
        # 無効化タスクIDデータの初期化
        for phase in dg_customize_config[0]._customize:
            if phase._subflow_type_name != 'plan':
                disable_task_ids_on_phase[phase._subflow_type_name] = []
            else:
                continue

        # DGカスタマイズプロパティの設定値とDGカスタマイズJSONデータから無効にするタスクIDを取得する
        for plan_property in plan_data['governance_plan']:
            if plan_property['is_enabled'] == False:
                # 無効なDGカスタマイズプロパティ（α項目）のIDを取得する
                alpha_id = plan_property['id']
                for alpha_config in dg_customize_config:
                    if alpha_config._id == alpha_id:
                        # DGカスタマイズプロパティ（α項目）のIDとDGカスタマイズ定義データIDが一致しているのみ処理
                        for subflow_rule in alpha_config._customize:
                            if subflow_rule._subflow_type_name != 'plan':
                                disable_task_ids_on_phase[subflow_rule._subflow_type_name].extend(subflow_rule._task_ids)

        return disable_task_ids_on_phase

    def disable_task_by_phase(self):
        """各サブフローの各タスクステータスのdisabledを更新"""
        disable_task_ids_data = self.get_disable_task_ids_on_phase()
        for phase, disable_task_ids in disable_task_ids_data.items():
            # data_gorvernance\base\subflow\<フェーズ>\status.jsonを更新する。
            status_path = os.path.join(self._abs_root_path, path_config.get_base_subflow_pahse_status_file_path(phase))

            sf = SubflowStatusFile(status_path)
            sub_flow_status:SubflowStatus = sf.read()
            for task in sub_flow_status._tasks:
                if task.id in disable_task_ids and not task.is_required:
                    # 無効化タスクIDリストに標的タスクIDが含まれ、かつ必須タスクではない場合、disabledを真にする
                    task.disable = True
                else:
                    task.disable = False
            sf.write(sub_flow_status)


    def change_submit_button_init(self, name):
        """処理メソッドボタンのメソッドです。

        Args:
            name (str): メッセージ
        """
        self.submit_button.name = name
        self.submit_button.button_type = 'primary'
        self.submit_button.button_style = 'solid'

    def change_submit_button_processing(self, name):
        """ボタンを処理中ステータスに更新するメソッドです。

        Args:
            name (str): 実行中のメッセージ
        """
        self.submit_button.name = name
        self.submit_button.button_type = 'primary'
        self.submit_button.button_style = 'outline'

    def change_submit_button_success(self, name):
        """ボタンが押されて成功した時のメッセージを返すメソッドです。

        Args:
            name (str): 成功したメッセージ
        """
        self.submit_button.name = name
        self.submit_button.button_type = 'success'
        self.submit_button.button_style = 'solid'

    def change_submit_button_warning(self, name):
        """ボタンが押されて認証が失敗した時の警告メッセージを返すメソッドです。

        Args:
            name (str): 警告メッセージ
        """
        self.submit_button.name = name
        self.submit_button.button_type = 'warning'
        self.submit_button.button_style = 'solid'

    def change_submit_button_error(self, name):
        """ボタンが押されて内部エラーが発生した時のエラーを返すメソッドです。

        Args:
            name (str): エラーメッセージ
        """
        self.submit_button.name = name
        self.submit_button.button_type = 'danger'
        self.submit_button.button_style = 'solid'


class DGPlaner(TaskDirector):
    """フェーズ：研究準備、タスク：研究データ管理計画を立てるのコントローラークラス
    
    Attributes:
        instance:
            working_path(str):実行Notebookファイルパス
            _plan_path(str):JSON定義書のパス
            form_box(pn.WidgetBox):フォームを格納する。
            msg_output(MessageBox):ユーザーに提示するメッセージを格納する。
    """

    def __init__(self, working_path:str) -> None:
        """DGPlaner コンストラクタのメソッドです

        Args:
            working_path (str): 実行Notebookファイルパス
        """
        # working_path = .data_gorvernance/researchflow/plan/task/plan/make_research_data_management_plan.ipynbが想定値
        super().__init__(working_path, notebook_name)

        # フォームボックス
        self.form_box = pn.WidgetBox()
        self.form_box.width = 900
        # メッセージ用ボックス
        self.msg_output = MessageBox()
        self.msg_output.width = 900

        # α設定JSON定義書(plan.json)
        # 想定値：data_gorvernance\researchflow\plan\plan.json
        self._plan_path =  os.path.join(self._abs_root_path, path_config.PLAN_FILE_PATH)

    @TaskDirector.task_cell("2")
    def generateFormScetion(self):
        """フォームセクション用"""
        # タスク開始による研究準備のサブフローステータス管理JSONの更新
        self.doing_task()

        # フォーム定義
        dg_customize = DGCustomizeSetter(self.nb_working_file_path, self._plan_path)
        dg_customize.define_form()

        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(dg_customize._form_box)
        form_section.append(dg_customize._msg_output)
        display(form_section)

DGPlaner(working_path=os.path.abspath('__file__')).generateFormScetion()

## Gakunin RDMに保存する
タスクの状態をGakunin RDMに保存します。

In [ ]:
# Gakunin RDMに保存する
import os
import sys
from pathlib import Path

import panel as pn
from IPython.display import display

sys.path.append('../../../../..')
from library.utils.config import path_config
from library.task_director import TaskDirector

script_file_name = "make_research_data_management_plan"
notebook_name = script_file_name+'.ipynb'

script_dir_path = os.path.dirname('__file__')
p = Path(script_dir_path)
# dmp.json(data_gorvernance\researchflow\plan\dmp.json)
dmp_file = p.joinpath('../../../../../researchflow/plan/dmp.json').resolve()

class DGPlaner(TaskDirector):
    """GRDMに保存するクラスです。

    Attributes:

        instance:
            working_path(str):実行Notebookファイルパス
    
    """

    def __init__(self, working_path:str) -> None:
        """DGPlaner コンストラクタ

        Args:
            working_path (str): 実行Notebookファイルパス
        """
        # working_path = .data_gorvernance/researchflow/plan/task/plan/make_research_data_management_plan.ipynbが想定値
        super().__init__(working_path, notebook_name)

    @TaskDirector.task_cell("3")
    def completed_task(self):
        """GRDMに保存するボタンの表示と、保存の実行を行うメソッドです。
        
        """
        base_subflow_path = os.path.join(self._abs_root_path, path_config.DG_SUB_FLOW_BASE_DATA_FOLDER)
        source = self.get_sync_source(path_config.STATUS_JSON, base_subflow_path)
        if os.path.isfile(dmp_file):
            source.append(str(dmp_file))
        # フォーム定義
        self.define_save_form(source)
        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self.save_form_box)
        form_section.append(self.save_msg_output)
        display(form_section)

    def get_sync_source(self, target_file_name: str, search_directory :str):
        """検索するファイル名のパスを取得するメソッドです。
        Args:
            target_file_name(str):見つけたいファイル名
            search_directory(str):検索するディレクトリ

        Returns:
            source(list):ファイル名のパス
        
        """

        source = []
        for root, dirs, files in os.walk(search_directory):
            for filename in files:
                if filename.startswith(target_file_name):
                    path = os.path.join(root, filename)
                    source.append(path)
        return source

DGPlaner(working_path=os.path.abspath('__file__')).completed_task()

## サブフローメニューを表示する

サブフローメニューへ遷移するボタンを表示します。

In [ ]:
# サブフローメニューを表示する
import os
import sys
sys.path.append('../../../../..')
from library.task_director import TaskDirector

script_file_name = "make_research_data_management_plan"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()